In [2]:
import sys
repo_path = '/home/micael.verissimo/paper_lzt/exp-connection-hep/'
sys.path.insert(0, repo_path)

In [2]:
import os
import json
import time
import pickle
import itertools

import numpy as np
import pandas as pd

from loguru import logger
from typing import List, Dict, Union
from src.constants import GeV, et_bins, eta_bins

import matplotlib.pyplot as plt 
import tensorflow as tf
from tensorflow.keras.models import Model
model_from_json = tf.keras.models.model_from_json

# Configurações do TensorFlow para evitar warnings
tf.config.run_functions_eagerly(True)
tf.data.experimental.enable_debug_mode()

# Configurações adicionais para melhor performance e menos warnings
tf.get_logger().setLevel('ERROR')  # Reduz logs verbosos
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'  # Suprime warnings de compilação

Welcome to JupyROOT 6.30/02


2025-07-14 15:24:21.614570: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-07-14 15:24:21.614608: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-07-14 15:24:21.632086: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-07-14 15:24:21.649363: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-07-14 15:24:29.788492: W tensorflow/compiler/tf2

In [3]:
def plot_stats(n_samples, save_path, output_name):
    '''
    '''
    etbins  = [f'[{l_et}, {h_et}[ GeV' for l_et, h_et in et_bins]
    etabins = [f'[{l_eta}, {h_eta}[' for l_eta, h_eta in eta_bins]
    # plot the samples
    fig, ax = plt.subplots(figsize=(15, 20))
    # Using matshow here just because it sets the ticks up nicely. imshow is faster.
    data = np.zeros((len(etbins), len(etabins)))
    ax.matshow(data, cmap='Greys')
    # anottate the samples
    for (i, j), z in np.ndenumerate(data):
        s, b = n_samples['et%i_eta%i' %(i,j)]
        s_b = (float(s)/float(b))*100
        ax.text(j, i-0.05, 'signal: %i' %s, ha='center', va='center', color='blue', fontsize=12)
        ax.text(j, i+0.05, 'background: %i' %b, ha='center', va='center', color='red', fontsize=12)
        ax.text(j, i+0.15, 's/b (%%): %1.2f' %(s_b), ha='center', va='center', color='black', fontsize=12)

    plt.axhline(y=1.5, lw=1.5, color='black')
    plt.axhline(y=0.5, lw=1.5, color='black')

    for ix in [0.5, 1.5, 2.5, 3.5]:
        plt.axvline(x=ix, lw=1.5, color='black')
    # change the bin labels
    ax.set_ylabel(r'$E_T$')
    ylabels = [''] + etbins + ['']
    ax.set_yticklabels(ylabels)

    ax.set_xlabel(r'$\eta$')
    xlabels = [''] + etabins + ['']
    ax.set_xticklabels(xlabels)

    plt.savefig(os.path.join(save_path, f'{output_name}.png'), dpi=200, facecolor='white', bbox_inches='tight')

In [4]:
trn_samples = {}
tst_samples = {}
for iet, (l_iet, h_iet) in enumerate(et_bins):
    for ieta, (l_ieta, h_ieta) in enumerate(eta_bins):
        print(f'Processing et bin = [{l_iet}, {h_iet}[ and eta bin = [{l_ieta}, {h_ieta}[')
        m_df.loc[(m_df.cl_et >= l_iet*GeV) & (m_df.cl_et < h_iet*GeV), 'et_bin'] = int(iet)
        m_df.loc[(m_df.cl_eta.abs() >= l_ieta) & (m_df.cl_eta.abs() < h_ieta), 'eta_bin'] = int(ieta)
        trn_samples[f'et{iet}_eta{ieta}'] = (len(m_df.loc[(m_df.et_bin == iet) & (m_df.eta_bin == ieta) & (m_df.target == 1)]),
                                             len(m_df.loc[(m_df.et_bin == iet) & (m_df.eta_bin == ieta) & (m_df.target != 1)]))
        
        t_df.loc[(t_df.cl_et >= l_iet*GeV) & (t_df.cl_et < h_iet*GeV), 'et_bin'] = int(iet)
        t_df.loc[(t_df.cl_eta.abs() >= l_ieta) & (t_df.cl_eta.abs() < h_ieta), 'eta_bin'] = int(ieta)
        tst_samples[f'et{iet}_eta{ieta}'] = (len(t_df.loc[(t_df.et_bin == iet) & (t_df.eta_bin == ieta) & (t_df.target == 1)]),
                                             len(t_df.loc[(t_df.et_bin == iet) & (t_df.eta_bin == ieta) & (t_df.target != 1)]))

Processing et bin = [15.0, 30[ and eta bin = [0.0, 0.8[


NameError: name 'm_df' is not defined

In [ ]:
m_df.cl_eratio.hist(bins=100)

In [ ]:
m_df.cl_reta.describe()

In [ ]:
m_df.cl_rhad.describe()

In [ ]:
m_df.cl_rhad.hist(bins=100)

In [ ]:
plot_stats(trn_samples, os.path.join(repo_path, 'data/processed'), 'train_samples')
plot_stats(tst_samples, os.path.join(repo_path, 'data/processed'), 'test_samples')

In [5]:
t_df = pd.read_parquet(os.path.join(repo_path, 'data/processed/lzt_zee_jf17_avg300_50k_classified.parquet'))

In [6]:
t_df.head()


,EventNumber,RunNumber,avgmu,cl_deta,cl_dphi,cl_e,cl_e0,cl_e1,cl_e2,cl_e233,...,v0_sigmoid,v0_linear,v0_tight_passed,v0_medium_passed,v0_loose_passed,v1_sigmoid,v1_linear,v1_tight_passed,v1_medium_passed,v1_loose_passed
0,40000,0,276.0,0.2,0.2,50539.238281,682.221191,40379.671875,9523.637695,14319.364258,...,1.0,13971.744141,1,1,1,1.0,45603.742188,1,1,1
3,40002,0,316.0,0.2,0.2,92922.898438,-33.155022,41428.152344,50405.484375,34472.343750,...,1.0,22548.000000,1,1,1,1.0,55837.890625,1,1,1
5,40040,0,342.0,0.2,0.2,83985.148438,-310.291595,64925.390625,19492.830078,18459.160156,...,1.0,115207.484375,1,1,1,1.0,59005.414062,1,1,1
6,40040,0,342.0,0.2,0.2,69624.156250,-301.798492,50327.515625,19697.140625,14645.372070,...,1.0,115207.484375,1,1,1,1.0,59005.414062,1,1,1
7,40041,0,231.0,0.2,0.2,48442.382812,233.507751,33527.433594,14191.050781,15718.147461,...,1.0,2353.416504,1,1,1,1.0,25257.417969,1,1,1


In [12]:
for iet, ieta in itertools.product(range(3), range(5)):
    print(f'Processing et bin = {iet} and eta bin = {ieta}')
    aux_df = t_df.loc[(t_df.et_bin == iet) & (t_df.eta_bin == ieta) & (t_df.target == 1)]
    tot = len(aux_df)
    for imodel in ['ss', 'v0', 'v1']:
        fig, ax = plt.subplots(figsize=(10, 6))
        ax.hist(aux_df[f'{imodel}_sigmoid'], bins=np.arange(0, 1.01, step=.01), label=f'{imodel} output', alpha=0.5)
        plt.savefig(os.path.join(repo_path, 'data/reports', 'figures', f'hist_{imodel}_et{iet}_eta{ieta}.png'), dpi=200, facecolor='white', bbox_inches='tight')
        plt.close()
        for iop in ['tight', 'medium', 'loose']:
            
            passed = len(aux_df.loc[aux_df[f'{imodel}_{iop}_passed'] == 1])
            print(f'  Model: {imodel}, OP: {iop}, Passed: {passed} ({(passed/tot)*100:.2f}%)')
        print(10*'~~')
    print(10*'-.-')
    print('\n')
    

Processing et bin = 0 and eta bin = 0
  Model: ss, OP: tight, Passed: 5198 (92.71%)
  Model: ss, OP: medium, Passed: 5345 (95.33%)
  Model: ss, OP: loose, Passed: 5532 (98.66%)
~~~~~~~~~~~~~~~~~~~~
  Model: v0, OP: tight, Passed: 5351 (95.43%)
  Model: v0, OP: medium, Passed: 5351 (95.43%)
  Model: v0, OP: loose, Passed: 5351 (95.43%)
~~~~~~~~~~~~~~~~~~~~
  Model: v1, OP: tight, Passed: 5601 (99.89%)
  Model: v1, OP: medium, Passed: 5601 (99.89%)
  Model: v1, OP: loose, Passed: 5601 (99.89%)
~~~~~~~~~~~~~~~~~~~~
-.--.--.--.--.--.--.--.--.--.-


Processing et bin = 0 and eta bin = 1
  Model: ss, OP: tight, Passed: 4050 (94.43%)
  Model: ss, OP: medium, Passed: 4130 (96.29%)
  Model: ss, OP: loose, Passed: 4239 (98.83%)
~~~~~~~~~~~~~~~~~~~~
  Model: v0, OP: tight, Passed: 4267 (99.49%)
  Model: v0, OP: medium, Passed: 4267 (99.49%)
  Model: v0, OP: loose, Passed: 4267 (99.49%)
~~~~~~~~~~~~~~~~~~~~
  Model: v1, OP: tight, Passed: 4272 (99.60%)
  Model: v1, OP: medium, Passed: 4272 (99.60%

In [11]:
et, eta = 2, 0

In [ ]:
print(list(t_df.columns))